In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import healpy as hp
import numpy as np
import h5py, os

from msfm.utils import files, catalog

### global constants

In [2]:
conf = files.load_config( "../configs/v16/default.yaml")

n_side = conf["analysis"]["n_side"]
n_pix = conf["analysis"]["n_pix"]
z_bins = conf["survey"]["metacal"]["z_bins"]

# relative to this notebook
pixels_file = "../" + conf["files"]["pixels"]
noise_file = "../" + conf["files"]["noise"]

# Load relevant properties from Metacalibration catalog

In [3]:
e1_tomo, e2_tomo, w_tomo = catalog.get_shapes_from_cat(conf)

26-02-16 08:37:10   catalog.py INF   Metacalibration bin 1: N_gal = 24940465 
26-02-16 08:37:23   catalog.py INF   Metacalibration bin 2: N_gal = 25280405 
26-02-16 08:37:29   catalog.py INF   Metacalibration bin 3: N_gal = 24891859 
26-02-16 08:37:36   catalog.py INF   Metacalibration bin 4: N_gal = 25091297 


# create the noise file

In [4]:
with h5py.File(noise_file, "w") as f:
    for z_bin, e1, e2, w in zip(z_bins, e1_tomo, e2_tomo, w_tomo):
        # shape (n_gals, 3)
        sample = np.stack([e1, e2, w], axis=1)

        # save as np.float32, single precision
        dset = f.create_dataset(name=f"{z_bin}/cat", data=sample, dtype="f")
        dset.attrs["info"] = "This dataset contains the ellipticities of the Metacalibration galaxy sample per redshift bin. " \
                             "The shape is (n_gals, 3), where axis 1 contains (e1, e2, w)."
